### 240721_note
- ~~point 설정까지는 ok.~~
- ~~문제1: cm 측정이 너무 민감하다~~
  - 잘 측정되고 있는지를 애초에, 어떻게 확인할 수 있을까?
  - 임의로 (initial ratio - ratio) * X * 50을 한다면, 그 적절한 값을 어떻게 찾을 수 있을까? (X = optimal_adjustment 변수) => 적당히 삽질하자
----
- 문제2: 특정 값에서 거북목임을 판단하는 기준이 없다
    -이건 애초에 판단할 필요가 없는가? 특정 값을 넘으면 그냥 움직여야 하니깐 => **추후 논의 필요할 듯** 
---
- 문제3: 위 로직으로 계산할 경우 턱을 드는 행위, 내리는 행위도 단순히 거리가 멀어진 것으로 판단한다. => 지금 보완 필요
---
- 문제4: 몸통 전체가 움직이는 경우에 대한 보완이 필요 (다같이 앞으로 가거나 / 다같이 뒤로 가면 distance를 구분하지 못한다)
---
- ~~보완1: 쓸데없는 코드 제거~~
- ~~보완2: 고개를 왼쪽으로/오른쪽으로 이동한 것에 대한 표시해주기 (각도는 절댓값으로, 방향을 함께)~~
- 보완3: 

### 1. 초기 설정 및 라이브러리 로드

In [13]:
import cv2
import mediapipe as mp
import numpy as np

# Mediapipe Face Mesh 및 Pose 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose

face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=1)

# OpenCV 비디오 캡처 초기화
cap = cv2.VideoCapture(0)



### 2. 랜드마크 인덱스 정의
- 얼굴 및 포즈의 랜드마크 정의

In [14]:

# 랜드마크 인덱스
chin_index = 152  # Face Mesh에서 턱 인덱스
left_cheek_index = 234  # 광대뼈 인덱스 (왼쪽)
right_cheek_index = 454  # 광대뼈 인덱스 (오른쪽)
left_shoulder_index = 11
right_shoulder_index = 12
nose_tip_index = 1
left_eye_index = 33
right_eye_index = 263

# 초기 랜드마크 위치
initial_face_landmarks = None
initial_shoulder_y = None
initial_ratio = None


### 3. 각도 및 거리 계산 함수 정의
- calculate_distance : 단순히 거리를 계산하는 함수
- calculate_ratios : 턱과 어깨선 사이의 거리와 광대 사이의 거리 비율을 계산하는 함수
- calculate_angle : 두 점 사이의 각도 계산, 얼굴에서 코-눈 중앙의 각도 변화를 계산하는데 사용한다.
- draw_landmark : facemesh 전체 나오면 더러워서 점 3개만 찍는 함수 만듬

In [15]:

# 거리 계산 함수
def calculate_distance(point1, point2):
    return np.linalg.norm(point1 - point2)

# 비율 계산 함수
def calculate_ratios(initial_face_points, current_face_points, initial_shoulder_y, current_shoulder_y):
    #어깨, 턱 좌표값 정의
    shoulder_center_y = (current_shoulder_y + current_shoulder_y) / 2
    chin_y = current_face_points[chin_index].y
    #어깨선과 턱선 y거리
    vertical_distance = chin_y - shoulder_center_y 
    #광대 랜드마크 행렬 정의 (연산을 위해서)
    left_cheek = np.array([current_face_points[left_cheek_index].x, current_face_points[left_cheek_index].y])
    right_cheek = np.array([current_face_points[right_cheek_index].x, current_face_points[right_cheek_index].y])
    horizontal_distance = calculate_distance(left_cheek, right_cheek) #얼굴 광대 사이의 거리 
    
    ratio = vertical_distance / horizontal_distance 
    return ratio

# 각도 계산 함수
def calculate_angle(point1, point2):
    return np.degrees(np.arctan2(point2[1] - point1[1], point2[0] - point1[0]))

# 코와 눈 사이의 y축 거리 계산 함수, 턱이 들렸는지 아닌지 확인하기 위하여
def calculate_nose_eye_distance(face_landmarks):
    nose = np.array([face_landmarks[nose_tip_index].x, face_landmarks[nose_tip_index].y])
    left_eye = np.array([face_landmarks[left_eye_index].x, face_landmarks[left_eye_index].y])
    right_eye = np.array([face_landmarks[right_eye_index].x, face_landmarks[right_eye_index].y])
    
    eye_center = (left_eye + right_eye) / 2
    nose_eye_distance = abs(nose[1] - eye_center[1])
    
    return nose_eye_distance


# 얼굴 3개 랜드마크 그리기 함수 
def draw_landmarks(image, landmarks):
    height, width, _ = image.shape
    
    # 턱 표시
    chin = landmarks[chin_index]
    chin_x = int(chin.x * width)
    chin_y = int(chin.y * height)
    cv2.circle(image, (chin_x, chin_y), 2, (0, 255, 0), -1)  # 초록색 원

    # 왼쪽 광대뼈 표시
    left_cheek = landmarks[left_cheek_index]
    left_cheek_x = int(left_cheek.x * width)
    left_cheek_y = int(left_cheek.y * height)
    cv2.circle(image, (left_cheek_x, left_cheek_y), 2, (0, 0, 255), -1)  # 빨간색 원

    # 오른쪽 광대뼈 표시
    right_cheek = landmarks[right_cheek_index]
    right_cheek_x = int(right_cheek.x * width)
    right_cheek_y = int(right_cheek.y * height)
    cv2.circle(image, (right_cheek_x, right_cheek_y), 2, (255, 0, 0), -1)  # 파란색 원

    # 눈코입 표시
    key_landmarks = [nose_tip_index, left_eye_index, right_eye_index]
    for idx in key_landmarks:
        x = int(landmarks[idx].x * width)
        y = int(landmarks[idx].y * height)
        cv2.circle(image, (x, y), 2, (0, 255, 0), -1)  # 초록색 원

# 각도 변화를 표현하는 함수
def format_angle_change(angle_change):
    direction = "right" if angle_change < 0 else "left"
    return f"{abs(angle_change):.2f} ({direction})"
    
# 거리 변화를 표현하는 함수
def format_distance_change(distance_change):
    direction = "forward" if distance_change < 0 else "backward"
    return f"{abs(distance_change):.2f} cm ({direction})"

### 4. 실행코드
- 카메라 프레임 획득

In [16]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_results = face_mesh.process(rgb_frame)
    pose_results = pose.process(rgb_frame)

    if face_results.multi_face_landmarks and pose_results.pose_landmarks:
        face_landmarks = face_results.multi_face_landmarks[0]
        pose_landmarks = pose_results.pose_landmarks

        face_landmarks = face_results.multi_face_landmarks[0]
        draw_landmarks(frame, face_landmarks.landmark)
        
        # 어깨 랜드마크 그리기 및 선으로 연결
        left_shoulder = pose_landmarks.landmark[left_shoulder_index]
        right_shoulder = pose_landmarks.landmark[right_shoulder_index]
        landmarks = [left_shoulder, right_shoulder]

        for landmark in landmarks:
            x = int(landmark.x * frame.shape[1])
            y = int(landmark.y * frame.shape[0])
            cv2.circle(frame, (x, y), 5, (255, 0, 0), -1)

        left_shoulder_pos = (int(left_shoulder.x * frame.shape[1]), int(left_shoulder.y * frame.shape[0]))
        right_shoulder_pos = (int(right_shoulder.x * frame.shape[1]), int(right_shoulder.y * frame.shape[0]))
        cv2.line(frame, left_shoulder_pos, right_shoulder_pos, (255, 0, 0), 2)

        if initial_face_landmarks is not None and initial_shoulder_y is not None:
            initial_face_landmarks = face_landmarks
            shoulder_y = (pose_landmarks.landmark[left_shoulder_index].y + pose_landmarks.landmark[right_shoulder_index].y) / 2 #y좌표의 평균값 
            ratio = calculate_ratios(initial_face_landmarks, face_landmarks.landmark, initial_shoulder_y, shoulder_y) #

            if initial_ratio is None:
                initial_ratio = ratio

            optimal_adjustment = 0.3 # 거리 측정을 위해, 조절할 필요가 있음
            distance_change = (initial_ratio - ratio) * 50 * optimal_adjustment   # 초기 거리를 기준으로 변환 => 이 결과가 정확한 cm를 반환하지 않는다 (너무 민감함) -> 적절한 optimal_adjustment를 추가할 필요가 있음.
        
            # 얼굴 회전 각도 계산 : initial_face_landmark.landmark 해주지 않으면 오류가 생긴다. 
            current_nose = np.array([face_landmarks.landmark[nose_tip_index].x, face_landmarks.landmark[nose_tip_index].y])
            current_left_eye = np.array([face_landmarks.landmark[left_eye_index].x, face_landmarks.landmark[left_eye_index].y])
            current_right_eye = np.array([face_landmarks.landmark[right_eye_index].x, face_landmarks.landmark[right_eye_index].y])
            current_eye_center = (current_left_eye + current_right_eye) / 2
            
            # 눈-코 거리 변화에 따른 텍스트 표시
            current_nose_eye_distance = calculate_nose_eye_distance(face_landmarks.landmark)
            if initial_nose_eye_distance is None:
                initial_nose_eye_distance = current_nose_eye_distance
                
            nose_eye_distance_change = current_nose_eye_distance - initial_nose_eye_distance
            nose_eye_direction = "up" if nose_eye_distance_change < 0 else "down"
            
            
            

            # initial angle이 None인 경우에만, s를 눌르면 초기화된다. 
            if initial_angle is None: 
                initial_nose = np.array([initial_face_landmarks.landmark[nose_tip_index].x, initial_face_landmarks.landmark[nose_tip_index].y])
                initial_left_eye = np.array([initial_face_landmarks.landmark[left_eye_index].x, initial_face_landmarks.landmark[left_eye_index].y])
                initial_right_eye = np.array([initial_face_landmarks.landmark[right_eye_index].x, initial_face_landmarks.landmark[right_eye_index].y])
                initial_eye_center = (initial_left_eye + initial_right_eye) / 2
    
                initial_angle = calculate_angle(initial_eye_center, initial_nose)
                initial_nose_eye_distance = calculate_nose_eye_distance(initial_face_landmarks)
                
            current_angle = calculate_angle(current_eye_center, current_nose)
            angle_change = current_angle - initial_angle

            # 텍스트 크기 및 위치 조정
            font_scale = 0.6  # 텍스트 크기 조정
            font_color = (0, 0, 0)  # 텍스트 색상 (검정색)
            line_type = 2
            
            # 텍스트 표시
            formatted_angle_change = format_angle_change(angle_change)
            formatted_distance_change = format_distance_change(distance_change)

            cv2.putText(frame, f'Face rotation: {formatted_angle_change}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_color, line_type)
            cv2.putText(frame, f'Distance: {formatted_distance_change}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_color, line_type)
            cv2.putText(frame, f'Current Ratio: {ratio:.2f}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_color, line_type)
            cv2.putText(frame, f'Tueck: {abs(nose_eye_distance_change):.2f} ({nose_eye_direction})', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_color, line_type)

        
        else:
            cv2.putText(frame, "Press 's' to set initial landmarks", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Live Stream', frame)

    key = cv2.waitKey(1) & 0xFF  

    if key == ord('s'):
        if face_results.multi_face_landmarks and pose_results.pose_landmarks:
            initial_face_landmarks = face_results.multi_face_landmarks[0].landmark
            initial_shoulder_y = (pose_results.pose_landmarks.landmark[left_shoulder_index].y + pose_results.pose_landmarks.landmark[right_shoulder_index].y) / 2
            initial_ratio = None  # 초기 비율 리셋

            # 초기 각도 설정
            initial_nose = np.array([initial_face_landmarks[nose_tip_index].x, initial_face_landmarks[nose_tip_index].y])
            initial_left_eye = np.array([initial_face_landmarks[left_eye_index].x, initial_face_landmarks[left_eye_index].y])
            initial_right_eye = np.array([initial_face_landmarks[right_eye_index].x, initial_face_landmarks[right_eye_index].y])
            initial_eye_center = (initial_left_eye + initial_right_eye) / 2
            initial_angle = calculate_angle(initial_eye_center, initial_nose)
            # 초기 거리 설정
            initial_nose_eye_distance = calculate_nose_eye_distance(initial_face_landmarks)
            print("Initial landmarks saved")


    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Initial landmarks saved
Initial landmarks saved
Initial landmarks saved
Initial landmarks saved
